# Introduction to Biopython
---
## R. Burke Squires
---
### NIAID Bioinformatics & Computational Biosciences Branch

This notebook is adapted from a Biopython Workshop given by Peter Cock [here](https://github.com/peterjc/biopython_workshop).

Adapted by: 
- R. Burke Squires (NIH/NIAID) richard.squires at niaid.nih.gov
- Carolyn Komatsoulis (NIH/NIAID) carolyn.komatsoulis at nih.gov

## What is Biopython?

1. Free, open source library for bioinformatics
1. Supported by Open Bioinformatics Foundation (OBF) along with BioPerl, BioJava, BioSQL
1. Runs on Mac OS X, Windows, Linux, etc.
1. International team of volunteer developers
1. Currently about three releases per year
1. Extensive "Biopython Tutorial & Cookbook"


## Python Package & Modules

1. Python module: A source code file (.py) which can contain:
    1. Classes
    1. Functions
    1. Global names (variables)
1. Python package: A directory of python module(s)

## Where Can I Find Details on Modules?

- Install using anaconda

    - **conda install biopython**
    

- In iPython or Python Interactive Shell: 

    - **from Bio import SeqIO**
    
    - **help(SeqIO)**                                              


- All Biopython modules

    - http://Biopython.org/DIST/docs/api/
    
    - "API" => application programmers interface

In [ ]:
!date


# Jumping into Biopython...

Lets download the sequence file we will be using but implementing some biopython code:

In [1]:
from Bio import Entrez
from Bio import SeqIO

Entrez.email='richard.squires@nih.gov' #please update with your email address
temp = Entrez.efetch(db="nucleotide", rettype="gb", id="NC_002549") 
gbseq = SeqIO.read(temp, "gb")

#open an output file and write data to it
out = open("ebola.gb",'w')
SeqIO.write(gbseq, out, "gb")

temp.close()
out.close()

#print(gbseq)
#print(gbseq.seq)

ImportError: No module named 'Bio'

Next lets convert the GenBank file to a FASTA file for us to use...

In [4]:
from Bio import SeqIO
count = SeqIO.convert("ebola.gb", "gb", "ebola.fa", "fasta")
print("You have converted %i file(s)." % count)

You have converted 1 file(s).


Now we want to extract the proteins translations from the GenBank file and save them to a new file:

In [7]:
from Bio import SeqIO
gbk_filename = "ebola.gb"
faa_filename = "ebola.faa"
input_handle  = open(gbk_filename, "r")
output_handle = open(faa_filename, "w")

for seq_record in SeqIO.parse(input_handle, "genbank") :
    print("Dealing with GenBank record %s" % seq_record.id)
    for seq_feature in seq_record.features :
        if seq_feature.type=="CDS" :
            assert len(seq_feature.qualifiers['translation'])==1
            output_handle.write(">%s from %s\n%s\n" % (
                   seq_feature.qualifiers['locus_tag'][0], #try also gene or product
                   seq_record.name,
                   seq_feature.qualifiers['translation'][0]))

output_handle.close()
input_handle.close()

Dealing with GenBank record NC_002549.1


## Bio - SeqIO: Counting Records

1. Count protein sequences in FASTA file, “ebola.fa”
2. Can use “grep’’ to count the number of proteins

        grep -c "^>" fasta.fa
        4141


3. Now let's count the records with Biopython using the "SeqIO.parse" function

4. Saved as “count_fasta.py” in workshop folder


## Bio - SeqIO: Count Records in a FASTA File

### Excersise:
    
    A. Modify this to count the number of records in the other FASTa files, both from E. Coli K12 and the potato genome ("PGSC_DM_v3.4_pep_representative.fasta")
   
    B. Using “sys.argv” get the filename as a command line argument, so that you can run it like this:
        a. python count_fasta_adv.py NC_000913.ffn


## Bio - SeqIO: Looking at the Sequence Records

1. "SeqIO.parse" function creates SeqRecord objects. 

2. Biopython's "SeqRecord" objects are a container holding the sequence, and any annotation about it - most importantly the identifier.

3. For FASTA files, the record identifier is taken to be the first word on the ">" line -- anything after a space is *not* part of the identifier.

4. This simple example prints out the record identifers and their lengths


In [ ]:
from Bio import SeqIO
filename = "ebola.faa"
for record in SeqIO.parse(filename, "fasta"):
    print("Record %s, length %i" % (record.id, len(record.seq)))


### Excercise 

    A. Count how many sequences are <100 amino acids long
    
    B. Create a modified script "total_length.py" based on the above examples which counts the number of records and calculates the total length of all the sequences (i.e. "21 + 820 + 310 + 428 + ... + 46 + 228"), giving:
        
        a. $ python total_length.py
        b. 4141 records, total length 1311442
    
    C. Plot a histogram of the sequence length distribution (tip - see the `Biopython Tutorial & Cookbook)




1. The "SeqRecord" objects the identifiers are stored as standard Python strings (e.g. ".id"). For the sequence, Biopython uses a string-like "Seq" object, accessed as ".seq".

2. In many ways the "Seq" objects act like Python strings, you can print them, take their length using the "len(...)" function, and slice them with square brackets to get a sub-sequence or a single letter.


## Bio - SeqIO: Record Lengths

1. Using "SeqIO.parse(...)" in a for loop, for each record print out the identifier, the first 10 letters of each sequences, the last 10 letters


In [ ]:
from Bio import SeqIO
filename = "ebola.fa"
for record in SeqIO.parse(filename, "fasta"):
    start_seq = record.seq[:10] # first 10 letters
    end_seq = record.seq[-10:] # last 10 letters
    print(record.id + " " + start_seq + "..." + end_seq)


## Bio - SeqIO: Check for Initial Methionine

1. How to check all the protein sequences start with a methionine (represented as the letter "M" in the standard IUPAC single letter amino acid code), and count how many records fail this.

In [ ]:
from Bio import SeqIO

filename = "ebola.faa"
bad = 0
for record in SeqIO.parse(filename, "fasta"):
    if not record.seq.startswith("M"):
        bad = bad + 1
        print(record.id + " starts " + record.seq[0])

print("Found " + str(bad) + " records in " + filename + " which did NOT start with M")

Excercise:

   1. Modify this script to print out the description of the problem records, not just the identifier. *Tip*: Try reading the documentation, e.g. Biopython's wiki page on the `SeqRecord <http://biopython.org/wiki/SeqRecord>`.


### Discussion: 

- What did you notice about these record descriptions? 
- Can you think of any reasons why there could be so many genes/proteins with a problem at the start?


## Bio - SeqIO: Check for Stop Codons

1. Let's check the example protein FASTA files for any "*" symbols in the sequence. For this you can use several of the standard Python string operations which also apply to "Seq" objects


In [ ]:
my_string = "MLNTCRVPLTDRKVKEKRAMKQHKAMIVALIVICITAVV\
            AALVTRKDLCEVHIRTGQTEVAVFTAYESE*"
my_string.startswith("M")

In [ ]:
my_string.endswith("*")

In [ ]:
len(my_string)

In [ ]:
my_string.count("M")

In [ ]:
my_string.count("*")

In [ ]:
from Bio import SeqIO
filename = "ebola.faa"
#filename = "PGSC_DM_v3.4_pep_representative.fasta"
contains_star = 0
ends_with_star = 0
print("Checking " + filename + " for terminal stop codons")
for record in SeqIO.parse(filename, "fasta"):
    if record.seq.count("*"):
        contains_star = contains_star + 1
    if record.seq.endswith("*"):
        ends_with_star = ends_with_star + 1
print(str(contains_star) + " records with * in them")
print(str(ends_with_star) + " with * at the end")

## Bio - SeqIO: Different File Formats:

1. If you work with finished genomes, you'll often see nicely annotated files in the EMBL or GenBank format. Let's try this with the *Ebola* GenBank file, "ebola.gb", based on the previous example:


In [ ]:
from Bio import SeqIO
fasta_record = SeqIO.read("ebola.fa", "fasta")
print(fasta_record.id + " length " + str(len(fasta_record)))

In [ ]:
genbank_record = SeqIO.read("ebola.gb", "genbank")
print(genbank_record.id + " length " + str(len(genbank_record)))

## Writing Sequence Files in Biopython

## Bio - SeqIO: Converting a Sequence File

1. Recall we looked at the *E. coli* K12 chromosome as a FASTA file "NC_000913.fna" and as a GenBank file "NC_000913.gbk". Suppose we only had the GenBank file, and wanted to turn it into a FASTA file?

2. Biopython's "SeqIO" module can read and write lots of sequence file formats, and has a handy helper function to convert a file.


In [ ]:
from Bio import SeqIO
help(SeqIO.convert)

In [ ]:
from Bio import SeqIO
input_filename = "ebola.gb"
output_filename = "ebola.fasta"
count = SeqIO.convert(input_filename, "gb", output_filename, "fasta")
print(str(count) + " records converted")

### Exercise: 
   
   1.Modify this to add command line parsing to take the input and output filenames as arguments


## Bio - SeqIO: Filtering a Sequence File: 

1. Suppose we wanted to filter a FASTA file by length, for example exclude protein sequences less than 100 amino acids long.

In [ ]:
from Bio import SeqIO
input_filename = "ebola.faa"
output_filename = "ebola_long_only.faa"
count = 0
total = 0
output_handle = open(output_filename, "w")
for record in SeqIO.parse(input_filename, "fasta"):
    total = total + 1
    if 500 <= len(record):
        count = count + 1
        SeqIO.write(record, output_handle, "fasta")
output_handle.close()
print(str(count) + " records selected out of " + str(total))

## Bio - SeqIO: Editing Sequences

1. Previous examples had a terminal "*” character (stop codon). Python strings, Biopython "Seq" and "SeqRecord" objects can all be *sliced* to extract a sub-sequence or partial record. In this case, we want to take everything up to but excluding the final letter:


In [ ]:
my_seq = "MTAIVIGAKILGIIYSSPQLRKCNSATQNDHSDLQISFWKDHLRQCTTNS*"
cut_seq = my_seq[:-1] # remove last letter
print(cut_seq)

### Exercise: 
    
1. Modify the following example to only remove the last letter if it is a "\*" (and save the original record unchanged if it does not end with "\*").


In [ ]:
from Bio import SeqIO
input_filename = "ebola.faa"
output_filename = "ebola_no_stars.faa"
output_handle = open(output_filename, "w")
for record in SeqIO.parse(input_filename, "fasta"):
    record = record[:-1]
    SeqIO.write(record,output_handle, "fasta")
output_handle.close()

## Bio - SeqIO: Filtering by Record Name

1. A very common task is pulling out particular sequences from a large sequence file. Membership testing with Python lists (or sets) is one neat way to do this.

2. Write a new script starting as follows which writes out the potato proteins on this list


In [ ]:
from Bio import SeqIO
wanted_ids = ["ZEBOVgp1", "ZEBOVgp3", "ZEBOVgp4"]
input_filename = "ebola.faa"
output_filename = "wanted_proteins.fasta"
count = 0
total = 0
output_handle = open(output_filename, "w")
for record in SeqIO.parse(input_filename, "fasta"):
    total = total + 1
    if record.id in wanted_ids:
        count = count + 1
        SeqIO.write(record, output_handle, "fasta")
output_handle.close()
print(str(count) + " records selected out of " + str(total))

### Advanced Exercise 

    A. Modify this to read the list of wanted identifiers from a plain text input file (one identifier per line).

### Discussion:

    A. What happens if a wanted identifier is not in the input file? 
    
    B. What happens if an identifier appears twice? 
    
    C. What order is the output file?


## Bio - SeqIO: Selecting by Record Name
1. What if you want the records in the specified order (regardless of the order in the FASTA file)?

2. In this situation, you can't make a single for loop over the FASTA file. For a tiny file you could load everything into memory (e.g. as a Python dictionary), but that won't work on larger files. 

3. Instead, we can use Biopython’s "SeqIO.index(...)" function which lets us treat a sequence file like a Python dictionary.


In [ ]:
from Bio import SeqIO
filename = "ebola.faa"
fasta_index = SeqIO.index(filename, "fasta")
print(str(len(fasta_index)) + " records in " + filename)
record = fasta_index["ZEBOVgp1"]
print(record)

In [ ]:
from Bio import SeqIO
wanted_ids = ["PGSC0003DMP400019313", "PGSC0003DMP400020381", "PGSC0003DMP400020972"]
input_filename = "PGSC_DM_v3.4_pep_representative.fasta"
output_filename = "wanted_potato_proteins_in_order.fasta"
fasta_index = SeqIO.index(input_filename, "fasta")
count = 0
total = len(fasta_index)
output_handle = open(output_filename, "w")
for identifier in wanted_ids:
    record = fasta_index[identifier]
    SeqIO.write(record, output_handle, "fasta")
    count = count + 1
output_handle.close()
print(str(count) + " records selected out of " + str(total))


## Bio - AlignIO: Reading Multiple - sequence Alignments
1. We're going to look at a small seed alignment for one of the PFAM domains, the `A2L zinc ribbon domain (A2L_zn_ribbon; PF08792). This was picked almost at random - it is small enough to see the entire alignment on screen, and has some obvious gap-rich columns.

2. From the alignments tab on the Pfam webpage, you can download the raw alignment in several different formats (Selex, Stockholm, FASTA, and MSF). Biopython is able to work with FASTA (very simple) and Stockholm format (richly annotated).


## Bio - AlignIO: Loading a Single Alignment:

1. As in "SeqIO", under "AlignIO" we have both
    
    A. "AlignIO.parse(...)" for looping over multiple separate alignments 

    B. "AlignIO.read(...)" for loading a file containing a single alignment

2. Most of the time you will be working with alignment files which contain a single alignment, so normally you will use "AlignIO.read(..)".

3. Here is an example loading the Pfam seed alignment for the `A2L zinc ribbon domain (A2L_zn_ribbon; PF08792):


In [9]:
from Bio import AlignIO
alignment = AlignIO.read("PF08792_seed.sth", "stockholm")
print(alignment)

FileNotFoundError: [Errno 2] No such file or directory: 'PF08792_seed.sth'

In many ways, the alignment acts like a list of "SeqRecord" objects (just like you would get from "SeqIO"). 


In [ ]:
print(len(alignment))

The length of the alignment is the number of rows for example, and you can loop over the rows as individual "SeqRecord" objects:


In [ ]:
for record in alignment:
    print(record.id + " has " + str(record.seq.count("-")) + " gaps")


Homework:
    
    A. Write a python script called "count_gaps.py" which reports the number or records, the total number of gaps, and the mean (average) number of gaps per record:


## Bio - AlignIO: Writing Multiple - sequence Alignment Files

1. As you might guess from using "SeqIO.convert(...)" and "SeqIO.write(...)", there are matching "AlignIO.convert()" and "AlignIO.write(...)" functions.

2. For example, this will convert the Stockholm formatted alignment into a relaxed PHYLIP format file:



In [ ]:
from Bio import AlignIO
input_filename = "PF08792_seed.sth”
output_filename = "PF08792_seed_converted.phy”
AlignIO.convert(input_filename, "stockholm", output_filename, "phylip-relaxed")


Homework: 

1. Modify this example to convert the Stockholm file into a FASTA alignment file.


This "AlignIO.convert(...)"  example is equivalent to using "AlignIO.read(...)" and "AlignIO.write(...)" explicitly:


In [ ]:
from Bio import AlignIO
input_filename = "PF08792_seed.sth”
output_filename = "PF08792_seed_converted.phy”
alignment = AlignIO.read(input_filename, "stockholm”)
AlignIO.write(alignment, output_filename, "phylip-relaxed")


This form is most useful if you wish to modify the alignment in some way, which we will do next.


## Bio - AlignIO: Sorting the Rows

How you can sort the rows by identifier within Biopython:


In [ ]:
from Bio import AlignIO
alignment = AlignIO.read("PF08792_seed.sth", "stockholm”)
alignment.sort()
print(alignment)

### Exercise:
    
1. Write a Python script "sort_alignment_by_id.py" which uses "AlignIO.read(..)" and "AlignIO.write(..)" to convert "PF08792_seed.sth" into a sorted FASTA file.
    
2. By default the alignment's sort method uses the identifers as the sort key, but much like how sorting a Python list works, you can override this.

3. Define your own function taking a single argument (a "SeqRecord") which returns the number of gaps in the sequence. Use this to sort the alignment and print it to screen (or save it as a new file)


## Sequence Features

## Sequence Features: Working with Sequence Features
1. Most of the time GenBank files contain a single record for a single chromosome or plasmid, so we'll generally use the "SeqIO.read(...)" function. Remember the second argument is the file format, so if we start from the code to read in a FASTA file


In [ ]:
from Bio import SeqIO
record = SeqIO.read("ebola.fa", "fasta")
print(record.id)

In [ ]:
from Bio import SeqIO
record = SeqIO.read("ebola.gb", "genbank")
print(record.id)
print(len(record))
print(len(record.features))


In [ ]:
my_gene = record.features[3]
print(my_gene)

Doing a print like this tries to give a human readable display. There are three key properties:
    
    1. ".type" which is a string like "gene" or "CDS"

    2. ".location" which describes where on the genome this feature is, and
    
    3. ".qualifiers" which is a Python dictionary full of all the annotation for the feature (things like gene identifiers).
    
This is what this gene looks like in the raw GenBank file::

gene     337..2799
	/gene="thrA”
	/locus_tag="b0002
	/gene_synonym="ECK0002; Hs; JW0001; thrA1; thrA2; thrD”
	/db_xref="EcoGene:EG10998”
	/db_xref="GeneID:945803”



## Sequence Features: Feature Locations

We're going to focus on using the location information for different feature types. Continuint with the same example:

In [ ]:
from Bio import SeqIO
record = SeqIO.read("ebola.gb", "genbank")
my_gene = record.features[3]
print(my_gene.qualifiers["locus_tag"])

In [ ]:
print(my_gene.location)

In [ ]:
print(my_gene.location.start)

In [ ]:
print(my_gene.location.end)

In [ ]:
print(my_gene.location.strand)

## Extracting Info from GenBank record

In [ ]:
from Bio import SeqIO
for index, record in enumerate(SeqIO.parse(open("ebola.gb"), "genbank")):
     print("index %i, ID = %s, length %i, with %i feat. " \
	% (index, record.id, len(record.seq), len(record.features)))


## BLAST

## A Few BLAST Details

(note: there was a picture on this slide. I need to figure that out)

## BLAST Output (Text)
    BLASTN 2.2.28+
    Reference: Zheng Zhang, Scott Schwartz, Lukas Wagner, and
    Webb Miller (2000), "A greedy algorithm for aligning DNA
    sequences", J Comput Biol 2000; 7(1-2):203-14.

    Reference for database indexing: Aleksandr Morgulis, George
    Coulouris, Yan Raytselis, Thomas L. Madden, Richa Agarwala,
    Alejandro A. Schaffer (2008), "Database Indexing for
    Production MegaBLAST Searches", Bioinformatics 24:1757-1764.

    RID: SJ2EFD07014

    Database: Nucleotide collection (nt)
               26,000,382 sequences; 49,159,429,833 total letters
    Query= gi|2765658|emb|Z78533.1| C.irapeanum 5.8S rRNA gene and ITS1 and ITS2
    DNA

    Length=740

                                                                 Score     E
    Sequences producing significant alignments:                       (Bits)  Value

    emb|Z78533.1|  C.irapeanum 5.8S rRNA gene and ITS1 and ITS2 DNA     1367   0.0   
    emb|FR720328.1|  Cypripedium irapeanum ITS1, 5.8S rRNA gene, I...   1210   0.0  


## BLAST Output (XML)

    <?xml version="1.0"?>
    <!DOCTYPE BlastOutput PUBLIC "-//NCBI//NCBI BlastOutput/EN" "http://www.ncbi.nlm.nih.gov/dtd/NCBI_BlastOutput.dtd">
    <BlastOutput>
      <BlastOutput_program>blastn</BlastOutput_program>
      <BlastOutput_version>BLASTN 2.2.28+</BlastOutput_version>
      <BlastOutput_reference>Zheng Zhang, Scott Schwartz, Lukas Wagner, and Webb Miller (2000), &quot;A greedy algorithm for aligning DNA sequences&quot;, J Comput Biol 2000; 7(1-2):203-14.</BlastOutput_reference>
      <BlastOutput_db>nr</BlastOutput_db>
      <BlastOutput_query-ID>gi|2765658|emb|Z78533.1|</BlastOutput_query-ID>
      <BlastOutput_query-def>C.irapeanum 5.8S rRNA gene and ITS1 and ITS2 DNA</BlastOutput_query-def>
      <BlastOutput_query-len>740</BlastOutput_query-len>
      <BlastOutput_param>
        <Parameters>
          <Parameters_expect>10</Parameters_expect>
          <Parameters_sc-match>1</Parameters_sc-match>
          <Parameters_sc-mismatch>-2</Parameters_sc-mismatch>
          <Parameters_gap-open>0</Parameters_gap-open>
          <Parameters_gap-extend>0</Parameters_gap-extend>
          <Parameters_filter>L;m;</Parameters_filter>
        </Parameters>
      </BlastOutput_param>
    <BlastOutput_iterations>
    <Iteration>
      <Iteration_iter-num>1</Iteration_iter-num>
      <Iteration_query-ID>gi|2765658|emb|Z78533.1|</Iteration_query-ID>
      <Iteration_query-def>C.irapeanum 5.8S rRNA gene and ITS1 and ITS2 DNA</Iteration_query-def>
      <Iteration_query-len>740</Iteration_query-len>
    <Iteration_hits>


## BLAST a Sequence to File

In [ ]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO

record = SeqIO.read(open("m_cold.fasta"), format="fasta")
result_handle = NCBIWWW.qblast("blastn", "nt", record.seq)
save_file = open("my_blast.xml", "w")
save_file.write(result_handle.read())
save_file.close()
result_handle.close()

## Parse BLAST Output

In [ ]:
from Bio.Blast import NCBIXML
result_handle = open("my_blast.xml")
blast_record = NCBIXML.read(result_handle)
E_VALUE_THRESH = 0.04
for alignment in blast_record.alignments:
     for hsp in alignment.hsps:
         if hsp.expect < E_VALUE_THRESH:
             print ('****Alignment****')
             print ('sequence:', alignment.title)
             print ('length:', alignment.length)
             print ('e value:', hsp.expect)
             print (hsp.query[0:75] + '...')
             print (hsp.match[0:75] + '...')
             print (hsp.sbjct[0:75] + '...')


## Next-Gen Related Scripts

[Source](http://Biopython.org/DIST/docs/tutorial/Tutorial.html#htoc217)

## FASTA, QUAL <=> FASTQ Conversion

Going from FASTQ to FASTA

In [ ]:
!wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR020/SRR020192/SRR020192.fastq.gz
!gunzip SRR020192.fastq.gz

In [ ]:
from Bio import SeqIO
SeqIO.convert("SRR020192.fastq", "fastq", "SRR020192.fasta", "fasta")

Going from FASTQ to QUAL is also easy:


In [ ]:
from Bio import SeqIO
SeqIO.convert("SRR020192.fastq", "fastq", "SRR020192.qual", "qual")

### FASTA and QUAL file to FASTQ:


In [ ]:
from Bio.SeqIO.QualityIO import PairedFastaQualIterator
for record in PairedFastaQualIterator(open("SRR020192.fasta"), open("SRR020192.qual")):
   print (record)


## Clean up FASTQ Files by Phred Score

In [ ]:
from Bio import SeqIO
good_reads = (rec for rec in \
	SeqIO.parse("SRR020192.fastq", "fastq") \
	if min(rec.letter_annotations["phred_quality"]) >= 20)
count = SeqIO.write(good_reads, "good_quality.fastq", "fastq")
print ("Saved %i reads" % count)

## Trimming Off Adaptor Sequences

In [ ]:
from Bio import SeqIO
def trim_adaptors(records, adaptor, min_len):
    len_adaptor = len(adaptor) 		# cache this for later
    for record in records:
        len_record = len(record) 	# Cache this for later
        if len(record) < min_len:	# Too short to keep
            continue
        index = record.seq.find(adaptor)
        if index == -1:		# Aadaptor not found, so won't trim
             yield record
        elif len_record - index - len_adaptor >= min_len:
            #after trimming this will still be long enough
            yield record[index+len_adaptor:]

original_reads = SeqIO.parse("SRR020192.fastq", "fastq")
trimmed_reads = trim_adaptors(original_reads, "GATGACGGTGT", 100)
count = SeqIO.write(trimmed_reads, "trimmed.fastq", "fastq") 
print ("Saved %i reads" % count)


## Working with NCBI Entrez

### ESearch: Searching the Entrez databases

To search any of these databases, we use Bio.Entrez.esearch(). For example, let’s search in PubMed for publications related to Biopython:

In [ ]:
from Bio import Entrez
Entrez.email = "richard.squires@nih.gov"     # Always tell NCBI who you are
handle = Entrez.esearch(db="pubmed", term="biopython")
record = Entrez.read(handle)
record["IdList"]

In [ ]:
from Bio import Entrez
Entrez.email = "richard.squires@nih.gov"     # Always tell NCBI who you are
handle = Entrez.efetch(db="nucleotide", id="186972394", rettype="gb", retmode="text")
print(handle.read())

## Biopython BioSQL Interface
1. Can use MySQL or Postgres Database

2. Must install and setup database software, database

3. Must load data into database

4. Python scripts in BioSQL folder (within biopython folder)


In [ ]:
from BioSQL import BioSeqDatabase
server = BioSeqDatabase.open_database(driver="MySQLdb", user="root", passwd = "", host = "localhost", db="bioseqdb")
db = server["orchids"]
for identifier in ['6273291', '6273290', '6273289'] :
    seq_record = db.lookup(gi=identifier)
    print(seq_record.id, seq_record.description[:50] + "...")
    print("Sequence length %i," % len(seq_record.seq))

## Additional Resources

# EMBOSS
1. European Molecular Biology Open Source Suite

    A.http://emboss.sourceforge.net

2.Command line programs to accomplish many bioinformatics tasks

3.Try out (for NIH access) on [Helix](http://helixweb.nih.gov/emboss/)

4.Biopython supports through [Bio.EMBOSS](http://Biopython.org/DIST/docs/api/Bio.Emboss-module.html)

## Resources: Python Programming

1. Websites

    A.http://wiki.python.org/moin/BeginnersGuide/NonProgrammers
    
    B.http://www.pythonforbeginners.com

2. Free eBook in HTML / PDF

    A.http://greenteapress.com/thinkpython/
    
    B.http://openbookproject.net/books/bpp4awd/index.html

3. Cheatsheets

    A. http://www.pythonforbeginners.com/cheatsheet/python-cheat-sheets/

4. Python Regular Expressions (pattern matching)

    A.http://www.pythonregex.com

5. Python Style Guide

    A.http://www.python.org/dev/peps/pep-0008/


## Goals
1. Introduce you to the basics of the Biopython package and some of the more popular Biopython modules

2. Enable you to find the information you need about Biopython 

3. Demonstrate how to apply Biopython to next-generation sequences data preparation

4. Enable you to write or assemble scripts of your own or modify existing scripts for your own purposes

5. Introduce you to EMBOSS software suite and ways to extend python and Biopython utilizing it.


## Q & A

Collaborations welcome

ScienceApps@niaid.nih.gov
